# Serialización de tipo no nativos

Pyro4 puede serializar todo lo que [pickle](https://docs.python.org/2/library/pickle.html) pueda serializar. Pero no todos los tipos de datos dentro de la librería estandard son serializables.

### Tipos de datos serializables por defecto
 * `bool`
 * `str`, `unicode`, `basestring`
 * `int`, `float`, `complex`
 * `set`, `frozenset`, `list`, `tuple`, `dict`

## Para que un tipo de dato no nativo sea serializable...

debe implementar [`__getstate__`](https://docs.python.org/2/library/pickle.html#object.__getstate__). En los los tipos de nativos que son privistos mediante [variaciones protegidas] (https://es.wikipedia.org/wiki/GRASP#Variaciones_Protegidas) como por ejemplo:



In [3]:
import os
stat = os.lstat('.')
print(stat)

posix.stat_result(st_mode=16877, st_ino=29865223, st_dev=16777221, st_nlink=15, st_uid=501, st_gid=20, st_size=510, st_atime=1445264635, st_mtime=1445264720, st_ctime=1445264720)


### Ya que Python es dinámico, podemos hacer [*monkeypatching*](https://en.wikipedia.org/wiki/Monkey_patch)?

Esta sería una solución sub-óptima, pero rápida...

In [4]:
def mi_getstate(self):
    return "algo intresante"
try:
    stat.__getstate__ = mi_getstate
except:
    print("No funcoina!")

No funcoina!


### Un wrapper podría ser una alternativa, pero que tal si la API de FUSE acepara algun tipo de dato naitvo?

Veamos el ejemplo que utiliza `os.lstat` del ejemplo de loopback.
```python
    def getattr(self, path, fh=None):
        st = os.lstat(path)
        return dict((key, getattr(st, key)) for key in ('st_atime', 'st_ctime',
            'st_gid', 'st_mode', 'st_mtime', 'st_nlink', 'st_size', 'st_uid'))
```
[Ver en Github](https://github.com/terencehonles/fusepy/blob/master/examples/loopback.py#L39)

Está utilizando un diccionario, en vez de devolver un tipo de datos personalizado.


### ¿Cómo generamos un diccionario a partir de un objeto?

### ¿Primero debemos saber de qué atributos se compone un objeto *x*?

Existe una función interna `dir()` que obtiene todos los atributos y métodos de un objeto, clase o módulo.

Veamos como funciona:

In [5]:
x = os.lstat('.')  # Como vimos más arriba esto nos devuelve un posix.stat_result
print(dir(x))

['__add__', '__class__', '__contains__', '__delattr__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getslice__', '__gt__', '__hash__', '__init__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'n_fields', 'n_sequence_fields', 'n_unnamed_fields', 'st_atime', 'st_birthtime', 'st_blksize', 'st_blocks', 'st_ctime', 'st_dev', 'st_flags', 'st_gen', 'st_gid', 'st_ino', 'st_mode', 'st_mtime', 'st_nlink', 'st_rdev', 'st_size', 'st_uid']


Los métodos que comienzan y terminan con doble guión bajo, son internos de Python y tienen un significado especial y no nos interesan, no son parte del ***estado*** del objeto.

### ¿Como obtenemos los nombres de los atributos y métodos de *x* que no son `__foo__`?

Las listas por comprenisón son una mecanismo de crear una lista describiendo sus características a partir de  un conjunto, en vez de enumerar sus elementos. Se utiliza la construcción **`for`**.

In [6]:
attrs = [name for name in dir(x) if '__' not in name]
print attrs

['n_fields', 'n_sequence_fields', 'n_unnamed_fields', 'st_atime', 'st_birthtime', 'st_blksize', 'st_blocks', 'st_ctime', 'st_dev', 'st_flags', 'st_gen', 'st_gid', 'st_ino', 'st_mode', 'st_mtime', 'st_nlink', 'st_rdev', 'st_size', 'st_uid']


#### O lo que es lo mismo, de una manera más funcional

In [5]:
attr_interesante = lambda a: not a.startswith('__')
atributos = lambda obj: filter(attr_interesante, dir(obj))
atributos(x)

['n_fields',
 'n_sequence_fields',
 'n_unnamed_fields',
 'st_atime',
 'st_birthtime',
 'st_blksize',
 'st_blocks',
 'st_ctime',
 'st_dev',
 'st_flags',
 'st_gen',
 'st_gid',
 'st_ino',
 'st_mode',
 'st_mtime',
 'st_nlink',
 'st_rdev',
 'st_size',
 'st_uid']

# Generando un diccionario

Además de `{'a': 1, 'b': 2}` y `dict(a=1, b=2)`, podemos crear un diccionario a partir de:
* *Lista de tuplas* `[('a', 1), ('b',  2)]` pasadas al `dict()` como argumento.
    * Es como se utiliza en el ejemplo de `loopback.py`
* *Por comprensión* con la notación `{k: v for k, v in [('a', 1), ('b', 2)]}`
    * Puede que ser que `v` sea una función de `k` y por lo tanto la iteración se hace sobre un iterable cuyos elmentos no son tuplas.

### Obtención de un atributo en Python

Python es un [lenguaje reflexivo](https://es.wikipedia.org/wiki/Reflexi%C3%B3n_%28inform%C3%A1tica%29), esto significa que se puede, dado un nombre, obtener el atributo de un objeto con que lleva ese nombre.


In [6]:
class Ejemplo(object):
    def __init__(self, nombre, edad):
        self.nombre = nombre
        self.edad = edad
o = Ejemplo("Foo", 12)
print(getattr(o, 'nombre'))
print(getattr(o, 'edad'))

Foo
12


### En el contexto de lstat...

In [7]:
def diccionario_de_atributos(cosa):
    return {attr: getattr(cosa, attr) for attr in dir(cosa) if not attr.startswith('__')}

In [8]:
diccionario_de_atributos(stat)

{'n_fields': 19,
 'n_sequence_fields': 10,
 'n_unnamed_fields': 3,
 'st_atime': 1445231158.0,
 'st_birthtime': 1437916096.0,
 'st_blksize': 4096,
 'st_blocks': 0,
 'st_ctime': 1445231134.0,
 'st_dev': 16777221,
 'st_flags': 0,
 'st_gen': 0,
 'st_gid': 20,
 'st_ino': 29865223,
 'st_mode': 16877,
 'st_mtime': 1445231134.0,
 'st_nlink': 14,
 'st_rdev': 0,
 'st_size': 476,
 'st_uid': 501}

![](static/meme_success.jpg)

# Probemos todo junto

In [3]:
from utils import lanzar_backgounrd, lanzar_como_proceso

In [4]:
lanzar_backgounrd('pyro4-ns', a_menos_que='lsof -t -i TCP:9090 -sTCP:LISTEN')

Lanzando pyro4-ns con pid 65785


(True, 65785)

In [5]:
import os
from Pyro4 import Daemon, Proxy
import socket

class Ejemplo(object):
    def lstat(self, path='.'):
        return os.lstat(path)

    def lstat_attr(self, path='.'):
        val = os.lstat(path)
        return diccionario_de_atributos(val)
    
def servidor():
    Daemon.serveSimple({
        Ejemplo(): 'ejemplo'
    }, ns=True, verbose=True, host=socket.gethostname())
    
with lanzar_como_proceso(servidor) as s:
    objeto_remoto = Proxy('PYRONAME:ejemplo')
    print(objeto_remoto.lstat_attr())

Lanzando <function servidor at 0x110a708c0> con el pid 65790.


Process Process-1:
Traceback (most recent call last):
  File "/usr/local/Cellar/python/2.7.10_2/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/2.7.10_2/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-c1b35034b8a9>", line 16, in servidor
    }, ns=True, verbose=True, host=socket.gethostname())
  File "/usr/local/lib/python2.7/site-packages/Pyro4/core.py", line 953, in serveSimple
    ns = Pyro4.naming.locateNS()
  File "/usr/local/lib/python2.7/site-packages/Pyro4/naming.py", line 435, in locateNS
    raise e
NamingError: Failed to locate the nameserver


NamingError: Failed to locate the nameserver